This notebook implements autoencoder trial run cycles.

* Define
    * trial name
    * environment to work with (MyBallsEnv, MyArmEnv, GymArmEnv)
    * dataset to use (set of 3 npz files for train, test and grid visualization, each file contains many angle/image pairs)
    * model builder
    * list of parameters to try
    * number of epochs to run 
    * output directory
* Run
    * fails if output directory exists, otherwise creates it
    * chooses a set of model parameters
Thursday 4/03/2021

* Visualize YYs (third output sheet or animated GIF) to assess decoder performance
* Rerun with [2, 10, 50] epochs
* Implement beta-VAE (see beta-vae notebook)

Next:

* Explore impact of changing filters chain down to 2x (2,2)
* Scatterplot 3D
* How does it fluctuate depending on network architecture, nlats, training protocol, etc
* Explore fold area in L0/L1 space. For each frame show:
 * image of gym robot,
 * colored L0/L1 scatterplot, with a red cross showing current latent state,
 * image reconstructed by the autoencoder

### imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

%load_ext tensorboard

import numpy as np
import os, datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.config import output_data_dir

### helpers

In [2]:
def train_models(models, datasets, params, tensorboard_logdir):
    ae = models['ae']
    
    tensorboard_callback = keras.callbacks.TensorBoard(tensorboard_logdir) #, histogram_freq=1)
    
    # FIXME ae.fit(dataset['Y'], validation_data=(val_dataset['Y'], val_dataset['Y']), callbacks=[tensorboard_callback],
    ae.fit(datasets['train']['images'], callbacks=[tensorboard_callback],
        epochs=params['epochs'], batch_size=128, verbose=0)

In [3]:
def cycle_autoencoder(ae, dataset, prefix="", N=10, vae=False, display=False):
    Y = dataset['images']
    
    if display:
        print("Y.shape=" + str(Y.shape))
    L = ae['enc'].predict(Y)
    L = np.array(L)
    if display:
        print("L.shape=" + str(L.shape))

    if vae:
        L = np.array(L)[2] # [z_mean, z_log_var, z] - take Z

    YY = ae['dec'].predict(L)
    YY = np.array(YY)
    if display:
        print("YY.shape=" + str(YY.shape))
    
    out_data = dict()
    out_data['latvars'] = L
    out_data['rec_images'] = YY

    # - Y
    if display:
        plot_Ys(Y[0:N,...], title="First %d elements of %sY" % (N, prefix))

        # - L
        fig, axs = plt.subplots(1, L.shape[1])
        fig.suptitle("Latent variables %s" % (str(L.shape)))
        for i in range(L.shape[1]):
            axs[i].hist(L[:,i])    

        # - YY
        plot_Ys(YY[0:N,...], title="First %d elements of %sYY" % (N, prefix))

        display_xvars(data)
    
    return out_data    

In [4]:
from src.models import save_models, save_models_weights
from src.visualization.lat import visualize_lat_space, animate_Y_YYs

def trial(dataset, ae_builder, params, suffix):
    print("*** trial(%s, %s)" % (str(params), suffix))
    
    vis_fname = "%s/%sa.png" % (IMGS_DIR, suffix)
    vis_fname2 = "%s/%sb.png" % (IMGS_DIR, suffix)
    ani_fname = "%s/%s.mp4" % (IMGS_DIR, suffix)
    latvars_fname = "%s/%s.csv" % (IMGS_DIR, suffix)
    
    if os.path.isfile(vis_fname):
        logger.error("%s exists, skipping ..." % vis_fname)
        return
    
    logger.info("training with params=%s" % str(params))
    keras.backend.clear_session()
    
    ae_models = ae_builder((64, 64, 1), 2, params)

    tensorboard_logdir = os.path.join(("%s/%s" % (TENSORBOARD_LOGS_DIR, suffix)),
                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    train_models(ae_models, dataset, params, tensorboard_logdir)

    save_models_weights(ae_models, "%s/%s" % (TRAINED_MODELS_DIR, suffix))
    
    logger.debug("running AE on grid dataset, saving visualization into %s" % (vis_fname))    
    dataset_grid_out = cycle_autoencoder(ae_models, dataset['grid'], vae=True, display=False)
    
    # save PNGs
    fig, axs = visualize_lat_space(dataset['grid'], dataset_grid_out, sheet=1)
    fig.savefig(vis_fname)
    fig.clf() # FIXME - not helping, figures still exist and slow things down
    
    fig, axs = visualize_lat_space(dataset['grid'], dataset_grid_out, sheet=2)
    fig.savefig(vis_fname2)
    fig.clf() # FIXME - not helping, figures still exist and slow things down

    # save MP4
    NANIFRAMES = 20
    logger.debug("saving ani visualization (for %d datapoints from grid dataset) into %s" % (NANIFRAMES, ani_fname))
    rng = np.random.default_rng()
    ani_is = rng.choice(range(dataset['grid']['images'].shape[0]), size=NANIFRAMES) # choose 20 random items from the dataset_grid
    animate_Y_YYs(dataset['grid']['images'][ani_is], dataset_grid_out['rec_images'][ani_is], outfile=ani_fname)
    
    # save CSV file
    angles_latvars = np.hstack((dataset['grid']['angles'], dataset_grid_out['latvars']))
    np.savetxt(latvars_fname, angles_latvars, delimiter=',')

### environment

In [5]:
import src.models.vae

# define ae builder method and environment name
ae_builder = src.models.vae.build_autoencoder
ENV_NAME = "twoballs"

### subdirectories

In [6]:
TRIAL_DIR = os.path.join(output_data_dir, ENV_NAME) # FUXNE

# fail if TRIAL_DIR exists
assert(not os.path.exists(TRIAL_DIR))
    
# create trial subdirs
TENSORBOARD_LOGS_DIR =  "%s/tensorboard-logs" % TRIAL_DIR
TRAINED_MODELS_DIR = "%s/trained-models" % TRIAL_DIR
#DATA_DIR = "%s/data" % TRIAL_DIR
IMGS_DIR = "%s/imgs" % TRIAL_DIR

#for dir in [TENSORBOARD_LOGS_DIR, TRAINED_MODELS_DIR, DATA_DIR, IMGS_DIR]:
for dir in [TENSORBOARD_LOGS_DIR, TRAINED_MODELS_DIR, IMGS_DIR]:
    os.makedirs(dir, exist_ok=True)

### load datasets

In [7]:
from src.data.dataset import load_datasets

dataset = load_datasets(ENV_NAME)

2021-03-07 00:34:44,709 - src.data.dataset - INFO - Loading from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_15000.npz ...
2021-03-07 00:34:45,228 - src.data.dataset - INFO - Loaded 15000 datapoints from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_15000.npz
2021-03-07 00:34:45,229 - src.data.dataset - INFO - Loading from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_1000.npz ...
2021-03-07 00:34:45,266 - src.data.dataset - INFO - Loaded 1000 datapoints from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_rand_1000.npz
2021-03-07 00:34:45,266 - src.data.dataset - INFO - Loading from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_grid_20_500.npz ...
2021-03-07 00:34:45,954 - src.data.dataset - INFO - Loaded 20000 datapoints from C:\Users\alexa\Documents\dvp\autoenc2\data\processed\twoballs_grid_20_500.npz


### train

In [ ]:
for epochs in [1, 2, 5, 10, 20, 50]:
    for i in range(3):
        output_suffix = "epochs_%d-%d" % (epochs, i)
        trial(dataset, ae_builder, {'epochs': epochs}, output_suffix)

### tensorboard

In [11]:
%tensorboard --logdir ..\data\output\twoballs

Reusing TensorBoard on port 6006 (pid 7788), started 0:01:52 ago. (Use '!kill 7788' to kill it.)